<a href="https://colab.research.google.com/github/Fernsrea/flow_matching/blob/main/MNIST_Discreet_FM_model_Inpainting_RandomMask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd ..

/content


In [1]:
!git clone https://github.com/Fernsrea/flow_matching.git
%cd flow_matching

Cloning into 'flow_matching'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 424 (delta 143), reused 73 (delta 73), pack-reused 220 (from 2)
Receiving objects: 100% (424/424), 3.43 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (187/187), done.
/content/flow_matching


In [2]:
%cd examples/image

/content/flow_matching/examples/image


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 70.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
!pip install torchdiffeq

In [ ]:
!pip install flow_matching

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torchvision import transforms
from PIL import Image

# Parameters
output_dir = "./data/image_generation/"
image_size = 28
mask_fraction = 0.5  # 50% pixels will be zeroed out
num_samples_to_plot = 5

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Load MNIST dataset
transform = transforms.ToTensor()
mnist = MNIST(root="./data", train=True, download=True, transform=transform)

# Function to create a random binary mask
def create_random_mask(size, fraction):
    total_pixels = size * size
    num_masked = int(total_pixels * fraction)
    mask = np.ones(total_pixels, dtype=np.uint8)
    mask[:num_masked] = 0
    np.random.shuffle(mask)
    return mask.reshape((size, size))

# Plotting lists
original_imgs = []
masked_imgs = []

# Apply random mask, save masked image, and collect samples
for idx, (img, _) in enumerate(mnist):
    img_np = (img.squeeze().numpy() * 255).astype(np.uint8)
    mask = create_random_mask(image_size, mask_fraction)
    masked_np = img_np * mask

    # Save masked image
    masked_img = Image.fromarray(masked_np)
    masked_img.save(os.path.join(output_dir, f"{idx:05d}.png"))

    # Collect some examples for plotting
    if idx < num_samples_to_plot:
        original_imgs.append(img_np)
        masked_imgs.append(masked_np)

    if idx % 1000 == 0:
        print(f"Saved {idx} masked images...")

In [ ]:
!python train.py \
    --dataset=mnist \
    --batch_size=32 \
    --accum_iter=1 \
    --eval_frequency=5 \
    --save_fid_samples \
    --num_workers=2 \
    --epochs=500 \
    --use_ema \
    --fid_samples=1000 \
    --discrete_flow_matching \
    --cfg_scale=0.0 \
    --data_path="./data/image_generation" \
    --output_dir='/content/drive/MyDrive/flow_matching_checkpoints_18M_RandomMask' \
    --compute_fid

Not using distributed mode
100% 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100% 28.9k/28.9k [00:00<00:00, 480kB/s]
100% 1.65M/1.65M [00:00<00:00, 3.87MB/s]
100% 4.54k/4.54k [00:00<00:00, 10.4MB/s]
/content/flow_matching/examples/image/training/grad_scaler.py:35: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self._scaler = torch.cuda.amp.GradScaler()
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:01<00:00, 95.4MB/s]
/content/flow_matching/examples/image/training/eval_loop.py:64: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(), torch.no_grad():


In [ ]:
!python train.py \
    --dataset=mnist \
    --batch_size=32 \
    --accum_iter=1 \
     --eval_frequency=5 \
    --save_fid_samples \
    --num_workers=2 \
    --epochs=500 \
    --use_ema \
    --fid_samples=1000 \
    --discrete_flow_matching \
    --cfg_scale=0.0 \
    --data_path="./data/image_generation" \
    --output_dir='/content/drive/MyDrive/flow_matching_checkpoints_18M_RandomMask' \
    --resume='/content/drive/MyDrive/flow_matching_checkpoints_18M_RandomMask/checkpoint-4.pth' \
    --compute_fid